## Copy the design files to the board

Rename and copy the BIT file and HWH file created by Vivado to a folder on your board. I renamed the files to:

* dma_axis_ip_relu.bit
* dma_axis_ip_relu.hwh



## Instantiate and download the overlay

In [ ]:
from pynq import Overlay

ol = Overlay("./dma_axis_ip_relu.bit")

check the IPs in this overlay using the IP dictionary (*ip_dict*).

In [ ]:
ol.ip_dict

The HLS has the default name from the Vivado project of *example_0*. 

Check *help* for the HLS IP:

In [ ]:
ol.example_0?

--- 
This tells us that this is not a known IP (type is **DefaultIP**) and will get assigned a default driver in PYNQ. The default driver provides MMIO read/write capability. 

## Create aliases

Using the labels for the HLS IP and DMA listed above

In [ ]:
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

hls_ip = ol.example_0 

## Check the status of the HLS IP

In [ ]:
hls_ip.register_map

## Start the HLS IP
Start the HLS IP by writing 0x81 to the control register. This will set bit 0 (AP_START) to "1" and bit 7 (AUTO_RESTART) to "1". AUTO_RESTART means the IP will run continuously. If we don't set this then after the IP completes one full operation or iteration, it will stop and wait until AP_START is set again. We would have to set this every time we want the IP to process some data.  

In [ ]:
CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

Check the correct bits have been set.

In [ ]:
hls_ip.register_map

## DMA send
Now we will send some data from DRAM to the HLS IP. Once the HLS IP is started, the steps are the same as the previous DMA tutorial. 

Note the array used below is uint32. This was selected to match the type of data width of the HLS IP and the widths used for the DMA. The DMA will transfer blocks of data and may "reformat" it based on the internal data widths in the hardware. The array we create in the notebook effects how the data is formatted in Python. This is an area that can cause problems so it is worthwhile checking you understand your data formats and data movement in your hardware. If you see the wrong 

In [ ]:
from pynq import allocate
import numpy as np

data_size = 1000
input_buffer = allocate(shape=(data_size,), dtype=np.int8)

Initialize the array. 

In [ ]:
for i in range(data_size):
    input_buffer[i] = i


### Print first few values of buffer 

In [ ]:
for i in range(10):
    print(hex(input_buffer[i]))

### Start the DMA transfer

In [ ]:
dma_send.transfer(input_buffer)

## DMA receive
Readback data from the HLS IP and store in DRAM. Start by creating the output buffer

In [ ]:
output_buffer = allocate(shape=(data_size,), dtype=np.int8)

In [ ]:
dma_recv.transfer(output_buffer)

### Print first few values of buffer 



In [ ]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

## Free all the memory buffers


In [ ]:
del input_buffer, output_buffer